In [ ]:
from karateclub import Graph2Vec

graph2vec = Graph2Vec(
    wl_iterations=80,
    attributed=True,
    dimensions=128,
    workers=4,
    epochs=5
)



In [8]:
from algorithm import GraphWrapper
from pathlib import Path
from tqdm import tqdm

input_path = Path('../benign_graphs/tc3-theia/firefox/nd')
input_paths = list(input_path.glob('*.json'))
input_graphs = [GraphWrapper(input_path) 
                for input_path in tqdm(input_paths, desc='Reading graphs')]


In [28]:
import networkx as nx
from algorithm.wrappers import IN, OUT
graph_path = Path('../benign_graphs/tc3-theia/firefox/nd/nd-9626934-processletevent.json')
graph = GraphWrapper(graph_path)
nxid_to_refid = []



G: nx.DiGraph = nx.DiGraph()
for node in graph.nodes:
    G.add_node(node.get_id(),
               token=node.get_token()
               )
for edge in graph.edges:
    G.add_edge(edge.node_ids[IN],
               edge.node_ids[OUT],
               token=edge.get_token()
               )